# Milestone #4

In this notebook we will look to fit a baseline model to the data. We first need severalfunctions to be able to split and analyze our data. First and foremost we will need to import sklearn packages in order to implement the model. We will also need to build some functions that will allow us to analyze the efficacy of particular models.

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn import cross_validation
from sklearn import metrics
from sklearn import ensemble
from sklearn import discriminant_analysis
import datetime

%matplotlib inline

In [2]:
def KFold_Score(folds, X, y, mdl, metric_func=metrics.accuracy_score):
    """
    Function to take in a single training set and check the efficacy 
    of a particular model using `folds`-fold validation.
    
    The function returns the mean of the `folds` `metric` scores
    """
    # fold the dataset into `folds`
    kf = cross_validation.KFold(len(X), n_folds=folds)
    
    # array to store results
    kf_res = np.empty((folds,))
    
    for i, (train_ix, test_ix) in enumerate(kf):
        fold_model = mdl
        fold_model.fit(X[train_ix, :], y[train_ix])
        
        # inputs are y_true, y_pred
        kf_res[i] = metric_func(y[test_ix], fold_model.predict(X[test_ix, :]))
        
    # aggregate scores by averaging 
    return np.nanmean(kf_res)

Let us load in our data.

In [3]:
FI = pd.read_csv('datasets/Food_Inspections.csv', index_col='Inspection ID')

In [4]:
FI.head()

,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
Inspection ID,,,,,,,,,,,,,,,,
1967170,"GOOSE ISLAND BAR - T1, B4",GOOSE ISLAND (T1-B4),2477070.0,Restaurant,Risk 2 (Medium),11601 W TOUHY AVE,CHICAGO,IL,60666.0,10/25/2016,License,Pass,NaN,42.008536,-87.914428,"(42.008536400868735, -87.91442843927047)"
1967164,ERMEL'S,ERMEL'S,2484238.0,Restaurant,Risk 1 (High),5729 N NORTHWEST HWY,CHICAGO,IL,60646.0,10/25/2016,License,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.986001,-87.787528,"(41.98600113711187, -87.78752792018707)"
1967146,"WENDY'S PROPERTIES, LLC",WENDY'S,2469194.0,Restaurant,Risk 1 (High),6324 N WESTERN AVE,CHICAGO,IL,60659.0,10/25/2016,License,Pass,NaN,41.996677,-87.689996,"(41.99667685737697, -87.68999575516291)"
1967133,LEARN TOGETHER GROW TOGETHER CHILD DEVELOPMENT...,LEARN TOGETHER GROW TOGETHER CHILD DEVELOPMENT C,2384887.0,Daycare Above and Under 2 Years,Risk 1 (High),1126 W 99TH ST,CHICAGO,IL,60643.0,10/25/2016,License,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.714241,-87.651464,"(41.714240500986136, -87.6514643961654)"
1967115,Porkchop,Porkchop,2373923.0,Restaurant,Risk 1 (High),29 E ADAMS ST,CHICAGO,IL,60603.0,10/24/2016,Short Form Complaint,Pass,NaN,41.879397,-87.626631,"(41.87939700209408, -87.62663087278814)"


### Data Cleaning

This data definitely needs to be cleaned. This will take several steps.

* Remove immediate non-predictor columns (`DBA Name`, `AKA Name`, `License #` (although this is useful later), `Address`)
* Remove uneccesary predictor columns (`City`, `State` (all in Chicago, IL), `Location` (already encapsulated in `Latitude`/`Longitude`)
* Remove (temporarily) inspection date. This will be useful when we add in data about weather
* Conversion of some columns into dummy variables easier for a computer to interpret.
    * `Facility Type` -> dummies (each separate)
    * `Risk` -> dummies (place on a scale, 1 highest etc.)
    * `Zip` -> dummies (each separate)
    * `Inspection Type` -> dummies (each separate)
    * `Violations` -> dummies (each separate),
* Extra Column for Number of `Violations`

We will need to abstract this whole process into a function so we can clean testing / OOS data.

We also need to consider the following:

In [5]:
len(FI['License #'].unique())

31097

In [6]:
len(FI.index.unique())

134192

The above means that in this dataset, many restaurants have been inspected more than once. What is unique to each restaurant is its `License #`.

The reason that this is a problem is that there would probably be some conditional distribution on past inspections and past inspection results. This will be a good thing to explore going forward. Does a failing grade on an inspection incentivize restaurants to clean up their act? Do restraunt who pass get complacent and relx their hygeine standards? 

In [7]:
FI.groupby('License #').last()

,DBA Name,AKA Name,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
License #,,,,,,,,,,,,,,,
0.0,"QuiteFrankly,Ltd.",UPS Cafeteria,Restaurant,Risk 1 (High),1400 S JEFFERSON ST,CHICAGO,IL,60607.0,01/06/2010,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.863501,-87.642210,"(41.86350135244771, -87.64220950077579)"
1.0,HARVEST CRUSADES MINISTRIES,HARVEST CRUSADES MINISTRIES,Special Event,Risk 2 (Medium),118 N CENTRAL AVE,CHICAGO,IL,60644.0,06/04/2010,Special Events (Festivals),Pass,NaN,41.882845,-87.765095,"(41.88284507471884, -87.76509545204392)"
2.0,COSI,COSI,Restaurant,Risk 1 (High),230 W MONROE ST,CHICAGO,IL,60606.0,06/15/2010,Canvass,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.880757,-87.634709,"(41.88075715864721, -87.6347092983425)"
9.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),116 S MICHIGAN AVE,CHICAGO,IL,60603.0,07/15/2010,Suspected Food Poisoning,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.880396,-87.624502,"(41.88039583825962, -87.62450172159464)"
40.0,COSI,COSI,Restaurant,Risk 1 (High),233 N MICHIGAN AVE,CHICAGO,IL,60601.0,08/23/2010,Canvass,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.886567,-87.624385,"(41.886567370886944, -87.62438467059714)"
43.0,COSI,COSI,NaN,Risk 3 (Low),28 E JACKSON BLVD,CHICAGO,IL,60604.0,05/04/2015,Canvass,Out of Business,NaN,41.878342,-87.626675,"(41.87834161206342, -87.62667499148682)"
62.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),230 W WASHINGTON ST,CHICAGO,IL,60606.0,06/21/2010,Canvass,Fail,"26. ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, P...",41.883318,-87.634769,"(41.88331785985083, -87.63476909461744)"
85.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),55 E GRAND AVE,CHICAGO,IL,60611.0,11/09/2010,Canvass,Fail,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.891591,-87.625867,"(41.891590741083505, -87.62586713724458)"
99.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,COSI,Restaurant,Risk 1 (High),203 N LA SALLE ST,CHICAGO,IL,60601.0,02/03/2010,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.885822,-87.632304,"(41.885822047853026, -87.63230373508567)"


However, for this baseline model, we will ignore these potential complications, although we undertand that delving into this will be an important part of future work.

Also, note that for our baseline model, we decompose the results into `Pass` or `Fail`. We hope to include multi-class classification into our final model too.

There are some further concerns too. For example, there are a great deal of Inspction Types. Let us see how many of them have been used. *NB:* A personal favourite is 'TWO PEOPLE ATE AND GOT SICK'.

In [8]:
FI['Inspection Type'].unique()

array(['License', 'Short Form Complaint', 'License Re-Inspection',
       'Complaint', 'Complaint Re-Inspection', 'Canvass',
       'Suspected Food Poisoning Re-inspection', 'Canvass Re-Inspection',
       'Suspected Food Poisoning', 'Tag Removal', 'Consultation',
       'Recent Inspection', 'Special Events (Festivals)', nan, 'Not Ready',
       'License-Task Force', 'Complaint-Fire Re-inspection',
       'Complaint-Fire', 'Short Form Fire-Complaint', 'Non-Inspection',
       'KITCHEN CLOSED FOR RENOVATION', 'O.B.', 'CORRECTIVE ACTION',
       'Package Liquor 1474', 'LICENSE CANCELED BY OWNER',
       'OWNER SUSPENDED OPERATION/LICENSE', 'LICENSE CONSULTATION',
       'License consultation', 'Task Force Liquor 1475',
       'Illegal Operation', 'fire complaint',
       'TWO PEOPLE ATE AND GOT SICK.', 'Pre-License Consultation',
       'CANVASS SPECIAL EVENTS', 'CANVASS SCHOOL/SPECIAL EVENT',
       'OUT OF BUSINESS', 'No entry', 'NO ENTRY', 'no entry',
       'TASK FORCE LIQUOR 1470', 

In [9]:
FI.groupby('Inspection Type').count().loc[:, 'License #'].sort_values(ascending=False).head(30)

Inspection Type
Canvass                                   70424
License                                   17610
Canvass Re-Inspection                     12835
Complaint                                 12266
License Re-Inspection                      6631
Short Form Complaint                       5329
Complaint Re-Inspection                    5061
Suspected Food Poisoning                    649
Consultation                                646
License-Task Force                          605
Tag Removal                                 603
Out of Business                             284
Task Force Liquor 1475                      254
Recent Inspection                           167
Complaint-Fire                              161
Suspected Food Poisoning Re-inspection      151
Short Form Fire-Complaint                   113
No Entry                                     60
Special Events (Festivals)                   56
Complaint-Fire Re-inspection                 44
Package Liquor 1474     

As we can see from the data above, the vast majority of `Inspection Type`s are confined to a small subset of the total number of types listed above. It would make sense to only use the most common data as columns for a dummy predictor and store the rest under the custom label of `misc`. The cutoff for this will be 100 registered inspection types. Although it would be more ideal to do this in a more rigorous automated, it is clear that we do not want information as to food poisoning reinspections masked by noise in the catch-all column. We do something very similar to the `Facility Type Column`

In [10]:
set(FI.groupby('Inspection Type').count().loc[:, 'License #'].sort_values(ascending=False).iloc[:17].index)

{'Canvass',
 'Canvass Re-Inspection',
 'Complaint',
 'Complaint Re-Inspection',
 'Complaint-Fire',
 'Consultation',
 'License',
 'License Re-Inspection',
 'License-Task Force',
 'Out of Business',
 'Recent Inspection',
 'Short Form Complaint',
 'Short Form Fire-Complaint',
 'Suspected Food Poisoning',
 'Suspected Food Poisoning Re-inspection',
 'Tag Removal',
 'Task Force Liquor 1475'}

In [11]:
import re

# helper functions abstracted for clarity, adaptibility
def results_helper(x):
    """
    Helper for results column
    """
    return np.where(x == 'Pass', 1, 0)

def inp_type_helper(df, col):
    """
    Helper for inspection type column.
    Would be great to have a better metric than 17 arbitrarily for the future.
    """
    dummy_set = set(df.groupby('Inspection Type').count().loc[:, 'License #'].sort_values(ascending=False).iloc[:17].index)
    return ['Misc' if x not in dummy_set else x for x in col]


def fac_type_helper(df, col):
    """
    Helper for inspection type column.
    Would be great to have a better metric than 17 arbitrarily for the future.
    """
    dummy_set = set(df.groupby('Facility Type').count().loc[:, 'License #'].sort_values(ascending=False).iloc[:21].index)
    return ['Misc' if x not in dummy_set else x for x in col]

def risk_helper(col):
    """
    Helper for risk column.
    Catch-all is 4
    """
    bad_set = ['All', np.nan]
    return [x.split(' ')[1] if x not in bad_set else 4 for x in col]


def viols_helper(df):
    """
    Helper for violations column.
    Also creates a column for number of vioaltions
    """
    # cleaned data. will be inputted into DF after cleaning
    viol_list_of_lists = []

    for i, viol in enumerate(df['Violations']):
        # for each establishment
        viols = []

        # if nan, no complaints
        if pd.isnull(viol):
            viol_list_of_lists.append(viols)
        else:
            # split into separate complaints
            viols = viol.split(' | ')
            for j, complaint in enumerate(viols):
                complaint = complaint.split(' - Comments: ')[0]
                viols[j] = complaint
            viol_list_of_lists.append(viols)
            
    violations_df = pd.Series([item for sublist in viol_list_of_lists for item in sublist])
    no_viols = [len(x) for x in viol_list_of_lists]
    
    for lst in viol_list_of_lists:
        for i, viol in enumerate(lst):
            code = viol.split('. ')[0]
            lst[i] = int(code)
            
    return (no_viols, viol_list_of_lists)
    
def clean_and_split(df):
    """
    Function to clean raw food inspection data and
    split this into predictor and label parts
    """
    df = df.drop(['DBA Name', 'AKA Name', 'Address', 'City', 'State', 'Location'], 1)
    df = df.drop('Inspection Date', 1) # NB will most likely be included in the final model
    
    # clean inspection types
    df['Inspection Type'] = inp_type_helper(df, df.loc[:, 'Inspection Type'])
    
    # clean facility types
    df['Facility Type'] = fac_type_helper(df, df.loc[:, 'Facility Type'])
    
    # clean risk types
    df['Risk'] = risk_helper(df['Risk'])
    
    # clean violations and add nnumber of violations
    no_viols, viols = viols_helper(df)
    df['# of Violations'] = no_viols
    df['Violations'] = viols
    
    # split columns into dummies
    viols_dummies_df = pd.get_dummies(pd.Series(df['Violations']).apply(pd.Series).stack()).sum(level=0)
    zip_dummies_df = pd.get_dummies(df['Zip'])
    inp_dummies_df = pd.get_dummies(df['Inspection Type'])
    fac_dummies_df = pd.get_dummies(df['Facility Type'])
    
    # drop columns that are now dummies
    df = df.drop(['Violations', 'Zip', 'Inspection Type', 'Facility Type'], 1)
    
    # add dummy columns
    df = pd.concat([df, viols_dummies_df, zip_dummies_df, inp_dummies_df, fac_dummies_df], axis=1)
    
    # drop last column
    df = df.drop('License #', 1)
    
    # drop nans, which will cause models to fail
    df = df.dropna(axis=0)
    
    # split off results and predictors and clean into Pass/Fail. Possible multiclass in future
    y = results_helper(df.loc[:, 'Results'])
    df = df.drop('Results', 1)
    
    return (df, y)

In [12]:
fi, y = clean_and_split(FI)

### Model Creation

This being a classification problem, let us see if we can tune a logistic regression model to this data.

In [13]:
from sklearn import linear_model

# baseline
KFold_Score(5, fi.as_matrix(), np.array(y), linear_model.LogisticRegression())

0.94330683827957507

In [14]:
def plot_tuning_results(tuning_vals, tuning_res_1, two_plots, tuning_res_2, log_flag, lab1, lab2, title):
    """
    Plot results for tuning parameters
    """
    plt.plot(tuning_vals, tuning_res_1, label=lab1, c='b')
    if two_plots:
        plt.plot(tuning_vals, tuning_res_2, label=lab2, c='g')
        
    plt.title(title)
    plt.xlabel('Tuning Values')
    plt.ylabel('Scores')
    
    if log_flag:
        plt.xscale('log')
        
    plt.ylim([0., 1.])
    plt.legend();

As we can see Logistic Regression is not necessarily the best model to use.

In [16]:
KFold_Score(5, fi.as_matrix(), np.array(y), ensemble.RandomForestClassifier())

0.94850345759129517

Using the Random Forest Classifier, we achieve a slightly better score. This can be attributable to the fact that tree ensembles do not expect linear features, which may not be present in the inspection data.

### Additional features (weather)

To improve the model further, we can aggregate external weather data to training data. We pulled data the daily max and daily minimum temperatures from weather stations in Chicago. After cleaning up the dataset, we appended the data to the entire inspection dataset.

In [66]:
weather_df = pd.read_csv('datasets/weather.csv')

In [67]:
weather_df.head()

,STATION,ELEVATION,LATITUDE,LONGITUDE,DATE,TAVG,TMAX,TMIN
0,GHCND:USC00111550,180.4,41.86611,-87.61528,20100101,-9999,21,10
1,GHCND:USC00111550,180.4,41.86611,-87.61528,20100102,-9999,16,7
2,GHCND:USC00111550,180.4,41.86611,-87.61528,20100103,-9999,24,6
3,GHCND:USC00111550,180.4,41.86611,-87.61528,20100104,-9999,21,13
4,GHCND:USC00111550,180.4,41.86611,-87.61528,20100105,-9999,27,19


In [68]:
weather_df['DATE'] = pd.to_datetime(weather_df['DATE'], format="%Y%m%d")

weather_df['Inspection Date'] = weather_df['DATE'].dt.strftime('%m/%d/%Y')

In [69]:
weather_df.drop(['STATION','LATITUDE','DATE','TAVG','ELEVATION','LONGITUDE'],inplace=True,axis=1)

In [70]:
weather_df.head()

,TMAX,TMIN,Inspection Date
0,21,10,01/01/2010
1,16,7,01/02/2010
2,24,6,01/03/2010
3,21,13,01/04/2010
4,27,19,01/05/2010


In [71]:
FI.set_index('Inspection Date').join(weather_df.set_index('Inspection Date'))

,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Type,Results,Violations,Latitude,Longitude,Location,TMAX,TMIN
Inspection Date,,,,,,,,,,,,,,,,,
01/02/2013,NICK'S GYROS,NICK'S GYROS,1403378.0,Restaurant,Risk 1 (High),2011 W 63RD ST,CHICAGO,IL,60636.0,Complaint,Pass,36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF...,41.779199,-87.674418,"(41.77919934204008, -87.67441843115479)",29,14
01/02/2013,NICK'S GYROS,NICK'S GYROS,1403378.0,Restaurant,Risk 1 (High),2011 W 63RD ST,CHICAGO,IL,60636.0,Complaint,Pass,36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF...,41.779199,-87.674418,"(41.77919934204008, -87.67441843115479)",30,11
01/02/2013,NICK'S GYROS,NICK'S GYROS,1403378.0,Restaurant,Risk 1 (High),2011 W 63RD ST,CHICAGO,IL,60636.0,Complaint,Pass,36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF...,41.779199,-87.674418,"(41.77919934204008, -87.67441843115479)",30,10
01/02/2013,NICK'S GYROS,NICK'S GYROS,1403378.0,Restaurant,Risk 1 (High),2011 W 63RD ST,CHICAGO,IL,60636.0,Complaint,Pass,36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF...,41.779199,-87.674418,"(41.77919934204008, -87.67441843115479)",30,10
01/02/2013,NICK'S GYROS,NICK'S GYROS,1403378.0,Restaurant,Risk 1 (High),2011 W 63RD ST,CHICAGO,IL,60636.0,Complaint,Pass,36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF...,41.779199,-87.674418,"(41.77919934204008, -87.67441843115479)",26,9
01/02/2013,NICK'S GYROS,NICK'S GYROS,1403378.0,Restaurant,Risk 1 (High),2011 W 63RD ST,CHICAGO,IL,60636.0,Complaint,Pass,36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF...,41.779199,-87.674418,"(41.77919934204008, -87.67441843115479)",26,6
01/02/2013,NICK'S GYROS,NICK'S GYROS,1403378.0,Restaurant,Risk 1 (High),2011 W 63RD ST,CHICAGO,IL,60636.0,Complaint,Pass,36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF...,41.779199,-87.674418,"(41.77919934204008, -87.67441843115479)",29,7
01/02/2013,NICK'S GYROS,NICK'S GYROS,1403378.0,Restaurant,Risk 1 (High),2011 W 63RD ST,CHICAGO,IL,60636.0,Complaint,Pass,36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF...,41.779199,-87.674418,"(41.77919934204008, -87.67441843115479)",30,11
01/02/2013,NICK'S GYROS,NICK'S GYROS,1403378.0,Restaurant,Risk 1 (High),2011 W 63RD ST,CHICAGO,IL,60636.0,Complaint,Pass,36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF...,41.779199,-87.674418,"(41.77919934204008, -87.67441843115479)",25,10


In [72]:
fi, y = clean_and_split(FI)

In [73]:
KFold_Score(5, fi.as_matrix(), np.array(y), ensemble.RandomForestClassifier())

0.94930075131680636

With the additional weather data, the accuracy score of our Random Forest model increases slightly. It should be noted that our weather data only takes the temperature from one weather station; to be even more accurate we can take the average of multiple weather station or perhaps indentify the one closest to the actual restaurant (using the latitude and longitude data).

### Additional features (nearby complaints)

In [17]:
from __future__ import division